### BLAST: Basic Local Alignment Search Tool

Consiste numa ferramenta para comparar sequências biológicas, conjunto de algoritmos e ferramentas para procurar sequências similares em bases de dados de grande dimensão. 

**Como funciona o [BLAST?](https://edu.taugc.com/blog/voce-sabe-como-o-blast-funciona/)**

O algoritmo identifica similaridades entre sequências por meio de correspondências curtas. De seguida, ocorre a procura por alinhamentos locais utilizando conjuntos de um tamanho definifo pelo utilizador, normalmente é de 3 letras para aminoácidos e 11 para nucleótidos. 

* Exemplo:
    * GLFKA seria analisada em conjuntos de três letras: GLK, LKF, KFA.

Os alinhamentos locais são construídos a partir de correspondências comuns, servindo como ponto de partida para uma extensão em ambas as direções. Essa extensão continua até que o alinhamento atinja um valor de pontuação pré-definido. 

Nesta UC, foi-nos proposto a criação de um algoritmo BLAST versão simplificada que assenta em três pontos principais:
* Considera apenas matches perfeitos entre a query e as sequências da BD
* Critérios simples usados para a extensão dos hits
* Score será a contagem do nº de matches

* **Query map**

A primeira função criada, **query_map()**, procura mapear as sequências através de uma janela de tamanho **w** para identificar as subsequências e os seus índices correspondentes. 

Inicialmente, a função verifica se a sequência de entrada é do tipo "ADN" ou "sequência de aminoácidos", utilizando a função auxiliar **tipo_seq()**. Em caso de não ser nenhum desses tipos, a função levanta um erro. Além disso, verifica se o valor inserido em w é um inteiro positivo válido. Após realizar um aprimoramento na sequência para prepará-la para a análise subsequente, a função divide a sequência em todas as possíveis sub-sequências de tamanho w e armazena-as numa lista.

É então criado um dicionário vazio para armazenar as subsequências e seus índices correspondentes. A função percorre a sequência e, para cada sub-sequência de tamanho w, verifica se ela já está presente no dicionário. Se estiver, adiciona o índice atual à lista de índices associada a essa sub-sequência. Se não estiver, cria uma nova entrada no dicionário com a sub-sequência como chave e uma lista contendo o índice atual como valor.

Finalmente, a função retorna o dicionário resultante, que contém as subsequências como chaves e os índices onde essas subsequências são encontradas na sequência inserida.

In [2]:
def query_map(query, w): 
    """
    Mapeia as sequências numa janela  para identificar subsequências

    
    Parâmetros:
    -------------
    query : str 
        Sequência a ser mapeada
    w : int 
        Tamanho da janela

        
    Retorna:
    -------------
    dict: Dicionário de subsequências mapeadas com seus índices.


    Levanta:
    -------------
    ValueError
        Caso sequência inserida seja inválida ou tamanho da janela não ser um inteiro positivo

        
    """

    from scripts.auxiliares import tipo_seq
    from scripts.auxiliares import aprimorar_seq

    if not isinstance(query, str):
        raise TypeError("A sequência deve ser uma string")
    if not isinstance(w, int) or w <= 0:
        raise ValueError("O tamanho da janela deve ser um inteiro positivo")

    
    if tipo_seq(query) not in ["DNA", "Sequência de aminoácidos"] :

        query = aprimorar_seq(query)

        seq_dic = {}

        for i in range(len(query) - w + 1):
            
            subsequence = query[i:i + w]

            if subsequence in seq_dic:

                seq_dic[subsequence].append(i)
            else:
                seq_dic[subsequence] = [i]

        return seq_dic

    else:
        raise ValueError("Sequência Inválida")

**Exemplo**:

In [3]:
query_map("ATCGCTG", 3)

{'ATC': [0], 'TCG': [1], 'CGC': [2], 'GCT': [3], 'CTG': [4]}

**Testes de unidade**:

In [4]:
import unittest

class TestBLASTFunctions(unittest.TestCase):
    def test_query_map_vazio(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            query_map("", 3)

    def test_w_vazio(self):
        # Testa se a função lança um erro ao receber um valor vazio para w
        with self.assertRaises(ValueError):
            query_map("ACT", "")

    def test_query_numerico(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            query_map(1, 3)
            
suite = unittest.TestLoader().loadTestsFromTestCase(TestBLASTFunctions)
unittest.TextTestRunner(verbosity=3).run(suite)

test_query_map_vazio (__main__.TestBLASTFunctions) ... ok
test_query_numerico (__main__.TestBLASTFunctions) ... ok
test_w_vazio (__main__.TestBLASTFunctions) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

* **Hits**

De seguida, foi criada uma função denominada **hits()** que realiza a análise dos hits entre as subsequências identificadas na sequência de consulta (query) e a sequência da base de dados.

A função recebe dois parâmetros:

* **dic_query**: Um dicionário proveniente da função anterior, query_map(), que mapeia as subsequências e seus índices na sequência de consulta.
* **seq**: A sequência da base de dados a ser analisada.

A função percorre cada entrada no dicionário, onde cada chave representa uma subsequência e o valor é uma lista de índices onde essa subsequência ocorre na sequência de consulta. Para cada valor na lista de índices, a função verifica se a chave (subsequência) está presente na sequência da base de dados (seq). Se estiver, utiliza expressões regulares para encontrar todas as ocorrências da subsequência na base de dados, obtendo uma lista de índices dessas ocorrências.

Em seguida, a função cria tuplos contendo os índices da subsequência na sequência de consulta e os índices correspondentes na sequência da base de dados. Esses tuplos são adicionados a uma lista.

Finalmente, a função retorna uma lista de tuplos, onde cada tuplo representa um hit entre a sequência de consulta e a sequência da base de dados, contendo os índices onde a subsequência foi encontrada em ambas as sequências.

In [5]:
def hits(qm : dict , seq : str) -> list: 

  """
  Devolve uma lista em que cada elemento é um tuplo com dois valores:
  1. O Offset na query
  2. O offset na seq

  Parâmetros
  ----------
  qm : dict 
    mapa de substrings que é devolvido ao invocar a função query_map
  
  seq : str
    sequência-alvo válida de DNA

  Returns
  -------
  res : list
    lista que contém um conjunto de tuplos que correspondem aos offsets correspondentes na query e sequência
  
  
  """

  res = []                                                # Lista de resultado que será populada com os tuplos

  for codon in qm:                                        # Cicla por todos os codons mapeados na query
    
    if codon in seq:         
                                                          # Apenas itera se o codon estiver na sequência
      for offset_q in qm[codon]:                          # Cicla por todos os offsets da query para cada codon 

        offset_seq = seq.find(codon)                      # Primeira instância do codon na sequência - devolve o índice

        while offset_seq != -1:                           # Enquanto for encontrada a instância do codon (o valor não é -1)
          
          res.append((offset_q,offset_seq))               # Adiciona um tuplo offset_query , offset_sequencia à lista de resultado
          # print(codon,'encontrado em', offset_seq)
          offset_seq = seq.find(codon, offset_seq + 1 )   # Procuramos pela próxima instância
  
  return res

**Exemplo**:

In [6]:
hits({'ATC': [0], 'TCG': [1], 'CGC': [2], 'GCT': [3], 'CTG': [4]}, "ATCGCGGG")

[(0, 0), (1, 1), (2, 2)]

**Testes de unidade**:

* **Extend_hit**


A função **extend_hit** desempenha um papel crucial na ampliação da correspondência entre duas sequências de DNA. Começa por receber quatro parâmetros: query (representando a sequência de DNA de procura), seq (representando a sequência alvo), hit (um dos elementos retornados pela função hits), e window (o tamanho da janela para criar dicionários de substrings na função query_map).

A primeira etapa consiste em extrair os elementos h1 e h2 do tuplo hit. Estes representam as posições iniciais na sequência de busca (query) e na sequência alvo (seq), respectivamente.

Em seguida, a função chama a função expande_dir duas vezes para ampliar a correspondência. A primeira chamada, para expandir para a esquerda, resulta em dois valores: tam_esq (o tamanho da expansão) e matches_esq (o número de matches corretos). A segunda chamada, para expandir para a direita, fornece tam_dt (tamanho da expansão) e matches_dt (número de matches corretos).

Por fim, a função retorna um tuplo contendo quatro informações cruciais:

* O offset inicial na sequência de busca (h1 - tam_esq)
* O offset inicial na sequência alvo (h2 - tam_esq)
* O tamanho total do resultado, levando em consideração a janela especificada (tam_esq + window + tam_dt)
* O número total de matches corretos, considerando a janela (matches_esq + window + matches_dt)

Dessa forma, a função extend_hit desempenha um papel vital na expansão e refinamento da correspondência inicial, proporcionando uma visão mais abrangente da similaridade entre as duas sequências de DNA.

In [7]:
def extend_hit(query : str , seq :str , hit : tuple, w : int) -> tuple: 
  
    """
    Estende um hit dado, identificando sua extensão na sequência de busca e na sequência alvo.

    
    Parâmetros
    ----------

    query : str 
      sequencia de DNA válida correspondente à sequência de busca

    seq   : str
      sequencia de DNA válida correspondente à sequência alvo
      
    hit   : tuple
      Um dos elementos devolvidos pela invocação da função hits

    w     : int
      o tamanho da janela para a criação de dicionários de substrings na função query map

    Retorna:
    --------
    tuple
      Devolve um tuplo com:
        1. O offset inicial na query
        2. O offset inicial na seq
        3. O tamanho do resultado
        4. O nº de matches corretos

    Levanta:
    XXXERROR
      Caso a sequência (seq) seja inválida    

    """
      
    h1 , h2 = hit
    
    tam_esq, matches_esq = expande_dir(query , seq , h1     , h2     , -1)

    tam_dt , matches_dt  = expande_dir(query , seq , h1 + w , h2 + w ,  1) 

    return (h1 - tam_esq , h2 - tam_esq , tam_esq + w + tam_dt , matches_esq + w + matches_dt)


**Exemplo**:

In [10]:
extend_hit("ATCG", "AAAATC", (1,1), 1 )

NameError: name 'expande_dir' is not defined

**Testes de unidade**:

* **Best_hit**

A função **best_hit** tem como objetivo identificar o hit mais próximo do início e o maior/mais preciso dentre todos os hits estendidos.

Para alcançar isso, a função inicia invocando a função **hits**, gerando uma lista de hits a partir do resultado da função query_map. Em seguida, utiliza a função extend_hit para cada hit na lista de hits estendidos, empregando o tamanho da janela fornecido como argumento.

A lista resultante de hits estendidos (extended_hits) é então percorrida, e a função mantém o hit com o maior número de matches corretos. O tuplo resultado é inicializado com zeros e é atualizado conforme se são encontrados hits mais precisos durante a iteração.

No final, a função retorna o melhor hit identificado, representado pelo tuplo resultante com o offset inicial na sequência query, o offset inicial na sequência alvo, o tamanho do resultado, e o número de matches corretos.

In [8]:
def best_hit(query : str , seq : str , window : int) -> tuple:
    
  """
  Itera sobre todos os hits extendidos e encontra o mais próximo do início
  e o maior / mais preciso
  
  Parâmetros
  ----------
  query : str 
    sequencia de DNA válida correspondente à sequência de busca

  seq   : str
    sequencia de DNA válida correspondente à sequência alvo

  w     : int
    o tamanho da janela para a criação de dicionários de substrings na função query map

  Returns:
  --------
  tuple
    Devolve um tuplo com:
      1. O offset inicial na query
      2. O offset inicial na seq
      3. O tamanho do resultado
      4. O nº de matches corretos
  """

  # Começamos pela lista de extended hits invocando a função:

  extended_hits = [extend_hit(query,seq,hit,window) for hit in hits(query_map(query,window),seq)]

  best_hit = (0,0,0,0)             # Iniciamos o tuplo-resultado

  for hit in extended_hits:        # Itera por todos os hits e mantém o que tem o match score maior
    if hit[3] > best_hit[3]:
        best_hit = hit
  
  return best_hit
    

**Exemplo**:

In [9]:
# RUI
# Para encontrar o melhor hit com base no maior numero de matches iteramos por todos os hits estendidos e vamos guardando o melhor
best_hit = (0,0,0,0)
for hit in extended_hits:
    if hit[3] > best_hit[3]:
        best_hit = hit
best_hit

NameError: name 'extended_hits' is not defined

**Testes de unidade**: